In [1]:
import numpy as np
from PIL import Image
import torchvision.transforms as transforms
import torch
import torchvision.models as models

# ---- 1. Load the precomputed features ----
landmark_features = np.load('/kaggle/input/feature-vector/landmark_features.npy')
non_landmark_features = np.load('/kaggle/input/feature-vector/non_landmark_features.npy')

# ---- 2. Define your feature extractor ----
# Assuming you're using a pretrained model like ResNet
import torchvision.models as models

# Use ResNet-50 to match the original feature dimension (2048)
model = models.resnet50(pretrained=True)
model.fc = torch.nn.Identity()
model.eval()


transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225]),
])

def extract_feature(image_path):
    img = Image.open(image_path).convert('RGB')
    img_tensor = transform(img).unsqueeze(0)  # shape: (1, 3, 224, 224)
    
    with torch.no_grad():
        feature = model(img_tensor).squeeze().numpy()
        feature = feature / np.linalg.norm(feature)  # Normalize
    return feature

# ---- 3. Define the classification function ----
def classify_image(new_image_path, landmark_features, non_landmark_features, threshold=0.7):
    new_feat = extract_feature(new_image_path)

    def cosine_sim(a, b):
        return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

    landmark_sims = np.dot(landmark_features, new_feat)
    non_landmark_sims = np.dot(non_landmark_features, new_feat)

    max_landmark_sim = landmark_sims.max() if landmark_sims.size > 0 else 0
    max_non_landmark_sim = non_landmark_sims.max() if non_landmark_sims.size > 0 else 0

    if max_landmark_sim > threshold and max_landmark_sim > max_non_landmark_sim:
        return "Landmark"
    elif max_non_landmark_sim > threshold:
        return "Non-Landmark"
    else:
        return "Unknown/Low similarity"

# ---- 4. Run the classifier on a test image ----
result = classify_image('/kaggle/input/test-landmark-images/isa_khan_color.jpg',
                        landmark_features, non_landmark_features)
print("Result:", result)

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 173MB/s]


Result: Landmark
